Katkı Sağlayanlar:
Emirhan Esim
Esma Kaya
Berkay Özçelik

In [105]:
import os,sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from PIL import Image, ImageOps
from skimage import io
from skimage import color
from skimage import transform
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from numpy import ndarray
from sklearn.model_selection import train_test_split 
import torch.utils.data as data_utils
import torchvision
import torch.optim as optim


In [106]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [107]:
#veri bilgilerinin bulunduğu csv dosyasını okuma
csv_path = r"C:\Users\emirh\Desktop\ai_hub_proje\data\UrbanSound8K\metadata\UrbanSound8K.csv"
csv_data = pd.read_csv(csv_path)
csv_data.tail(5)

,slice_file_name,fsID,start,end,salience,fold,classID,class
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn
8731,99812-1-6-0.wav,99812,332.289233,334.821332,2,7,1,car_horn


In [108]:
#görüntünün veri yolunu input olarak alır ve sırasıyla;
#grayscale dönüşümü, resizing, normalization, [görüntü, etiket] formatına yazma
#işlemlerini yapar.
#[görüntü,etiket] formatında DataFrame'i oluşturur ve csv dosyası olarak kaydını yapar 


def preprocessor(directory):
    images = []
    labels = []

    for foldername in os.listdir(directory): #spectrograms klasöründeki classID pathlerini döndür
        f = os.path.join(directory, foldername) #classID klasörlerinin pathleri
        for imagename in os.listdir(f):
            imagepath = os.path.join(f, imagename)
            rgb_img = io.imread(imagepath)[:,:,:3]
            gray_img = color.rgb2gray(rgb_img)
            gray_img_resized = transform.resize(gray_img, (128,128)) #grayscaled fotoğraf
            image = np.array(gray_img_resized, 'float32')
            image = (image-image.mean())/image.std()
            label = int(foldername)    
            images.append(image)
            labels.append(label)
    return images, labels
    

In [109]:
directory = r'C:\Users\emirh\Desktop\ai_hub_proje\data\spectrograms'

images, labels = preprocessor(directory)

In [110]:
len(images), len(labels)

(8732, 8732)

In [111]:
images[0]

array([[ 3.6713247 , -0.28672266, -1.8196371 , ..., -1.8227298 ,
        -1.8227298 , -1.8227298 ],
       [ 3.677114  , -0.27035457, -1.8173009 , ..., -1.8204195 ,
        -1.8204175 , -1.8204162 ],
       [ 3.6771352 , -0.27029043, -1.8172846 , ..., -1.8203738 ,
        -1.8129799 , -1.8074372 ],
       ...,
       [ 3.6768723 , -0.20892416, -0.86748713, ...,  1.884726  ,
         1.6924425 ,  0.9927972 ],
       [ 3.7136507 ,  1.0344989 ,  0.2612872 , ...,  0.9487419 ,
         0.8893153 ,  0.67563593],
       [ 3.8196363 ,  3.7934604 ,  3.7889116 , ...,  3.7891667 ,
         3.7891512 ,  3.7890959 ]], dtype=float32)

In [112]:
images[0]

array([[ 3.6713247 , -0.28672266, -1.8196371 , ..., -1.8227298 ,
        -1.8227298 , -1.8227298 ],
       [ 3.677114  , -0.27035457, -1.8173009 , ..., -1.8204195 ,
        -1.8204175 , -1.8204162 ],
       [ 3.6771352 , -0.27029043, -1.8172846 , ..., -1.8203738 ,
        -1.8129799 , -1.8074372 ],
       ...,
       [ 3.6768723 , -0.20892416, -0.86748713, ...,  1.884726  ,
         1.6924425 ,  0.9927972 ],
       [ 3.7136507 ,  1.0344989 ,  0.2612872 , ...,  0.9487419 ,
         0.8893153 ,  0.67563593],
       [ 3.8196363 ,  3.7934604 ,  3.7889116 , ...,  3.7891667 ,
         3.7891512 ,  3.7890959 ]], dtype=float32)

In [113]:
labels[0]

0

In [123]:
#train ve test verilerini bölme
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.15, random_state=42)

In [124]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

7422 7422
1310 1310


In [125]:
#validation verisi eldesi
X_val = X_train[6112:]
y_val = y_train[6112:]
X_train = X_train[:6112]
y_train = y_train[:6112]

In [126]:
print(f" X_train: {len(X_train)}, y_train: {len(y_train)} ")
print(f" X_val: {len(X_val)}, y_val: {len(y_val)} ")
print(f" X_test: {len(X_test)}, y_test: {len(y_test)} ")

 X_train: 6112, y_train: 6112 
 X_val: 1310, y_val: 1310 
 X_test: 1310, y_test: 1310 


In [127]:
#numpy array çevrimi
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)
X_val_np = np.array(X_val)
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)
y_val_np = np.array(y_val)

In [128]:
print(X_train_np.shape)
print(X_test_np.shape)
print(X_val_np.shape)
print(y_train_np.shape)
print(y_test_np.shape)
print(y_val_np.shape)

(6112, 128, 128)
(1310, 128, 128)
(1310, 128, 128)
(6112,)
(1310,)
(1310,)


In [131]:
#numpy arrayler tensora çevriliyor
X_train_t = torch.from_numpy(X_train_np).unsqueeze(-1).permute(0,3,1,2)
X_test_t = torch.from_numpy(X_test_np).unsqueeze(-1).permute(0,3,1,2)
X_val_t = torch.from_numpy(X_val_np).unsqueeze(-1).permute(0,3,1,2)
y_train_t = torch.from_numpy(y_train_np)
y_test_t = torch.from_numpy(y_test_np)
y_val_t = torch.from_numpy(y_val_np)

In [132]:
X_train_t.shape, X_test_t.shape, X_val_t.shape

(torch.Size([6112, 1, 128, 128]),
 torch.Size([1310, 1, 128, 128]),
 torch.Size([1310, 1, 128, 128]))

In [133]:

y_train_t.shape, y_test_t.shape, y_val_t.shape

(torch.Size([6112]), torch.Size([1310]), torch.Size([1310]))

In [134]:
#tensorleri klasöre kayıt etme
os.chdir(r"C:\Users\emirh\Desktop\ai_hub_proje\tensors")
torch.save(X_train_t, 'X_train_t.pt')
torch.save(X_test_t, 'X_test_t.pt')
torch.save(X_val_t, 'X_val_t.pt')
torch.save(y_train_t, 'y_train_t.pt')
torch.save(y_test_t, 'y_test_t.pt')
torch.save(y_val_t, 'y_val_t.pt')